In [ ]:
%matplotlib inline
import ipyvolume as ipv
import numpy as np
from scipy.ndimage.filters import convolve as conv
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
from skimage import feature

np.seterr(divide='ignore', invalid='ignore')

def generate_spheres_test_volume(size, num_beads):
    data = np.zeros((size, size, size))
    spheres = np.random.randint(5, size-5, size=(3, num_beads))
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            for z in range(data.shape[2]):
                if np.any(np.linalg.norm(spheres.T-[x, y, z], axis=1) < 4.8):
                    data[x, y, z] = 1
                    
    return data

def deconv_rl(data, iterations, blur):
    estimate = 0.5 * np.ones_like(data)
    for i in range(iterations):
        print("It: {}".format(i))
        estimate = estimate * blur(data / blur(estimate))
    
    return estimate

def extract_area(data, point, size):
    data = np.pad(data, ((size, size), (size, size), (size, size)), mode='constant', constant_values=0)
    point = np.add(point, (size, size, size))
    return data[int(point[0]-size):int(point[0]+size), int(point[1]-size):int(point[1]+size), int(point[2]-size):int(point[2]+size)]

def gaussian_3d(x, A, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z):
    return (A * np.exp(-(
        (x[0]-mu_x)**2 / (2*sigma_x**2) +
        (x[1]-mu_y)**2 / (2*sigma_y**2) +
        (x[2]-mu_z)**2 / (2*sigma_z**2)
    )))

def fit_gaussian(data: np.ndarray):
    from scipy.optimize import curve_fit
    xs = np.arange(data.shape[0])
    ys = np.arange(data.shape[1])
    zs = np.arange(data.shape[2])
    xv, yv, zv = np.meshgrid(xs, ys, zs)
    xv = xv.ravel()
    yv = yv.ravel()
    zv = zv.ravel()
    xdata = np.vstack((xv, yv, zv))

    # start = [data.shape[0]//2, data.shape[1]//2, data.shape[2]//2, r, r, r]
    upper_bounds = [np.inf, data.shape[0], data.shape[1], data.shape[2], np.inf, np.inf, np.inf]
    p0 = (1, data.shape[0]//2, data.shape[1]//2, data.shape[2]//2, 1, 1, 1)
    popt, _ = curve_fit(gaussian_3d, xdata, data[xv, yv, zv], bounds=(np.zeros((7, )), upper_bounds), p0=p0)

    # print(popt[4:])
    return popt[4:]

STATIC_VOL = True

# psf = np.ones((10, 10, 10)) / 10**3
psf = np.zeros((13, 13, 13))
psf[6, 6, 6] = 1
psf = gaussian_filter(psf, (1, 1, 1))

# de psf

In [ ]:
ipv.quickvolshow(psf)

In [ ]:
vol = generate_spheres_test_volume(55, 12)
vol = np.pad(vol, 9, 'constant', constant_values=0)
vol[5, 5, 5] = 1

# het gegenereerde input volume

In [ ]:
ipv.quickvolshow(vol)

In [ ]:
blurred_vol = conv(vol, psf)

# het geblurrede volume

In [ ]:
ipv.quickvolshow(blurred_vol)

# De bead detecteren en de sigma estimeren

In [ ]:
blobs = feature.blob_log(blurred_vol[:10, :10, :10], threshold=.001)

In [ ]:
blob_area = extract_area(blurred_vol, blobs[0, :3], 5)
est_sigma = fit_gaussian(blob_area)

In [ ]:
noisy_blurred_vol = blurred_vol + (np.random.poisson(lam=25, size=blurred_vol.shape) - 10) / 255.

# Het volume met noise

In [ ]:
ipv.quickvolshow(noisy_blurred_vol, level=[0.17, 0.50, 0.90], opacity=[0.01, 0.05, 0.1], data_min=0, data_max=1)

# Deconvolven met de gestimeerde sigma

In [ ]:
blur = lambda x: gaussian_filter(x, est_sigma)
result = deconv_rl(noisy_blurred_vol, 20, blur)
result = result.clip(-0.5, 1.18)
result = (result-result.min())/(result.max()-result.min())

In [ ]:
ipv.quickvolshow(result, level=[0.17, 0.50, 0.90], opacity=[0.01, 0.05, 0.1], data_min=0, data_max=1)

# Het verschil tussen het origineel en het gedeconvolvede resultaat

In [ ]:
diff = np.abs(vol - result)

In [ ]:
ipv.quickvolshow(diff, level=[0.17, 0.50, 0.90], opacity=[0.01, 0.05, 0.1], data_min=0, data_max=1)